In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import random
import math 
import copy
import random
import argparse
import torch.optim as optim
import torch.nn as nn
import numpy as np
from scipy.optimize import minimize
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import numpy as np

cuda:0


In [2]:
import torch
import torch.nn as nn

# 定义一个简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu2 = nn.ReLU()

        # 使用Xavier初始化来初始化权重
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)

        # 初始化偏置为零
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.constant_(self.fc2.bias, 0)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        return out
        
# 定义输入、隐藏层和输出的维度
input_size = 10
hidden_size = 20
output_size = 10

# 创建一个SimpleNN模型



In [3]:
class customloss(nn.Module):
    def __init__(self, theta):
        super(customloss, self).__init__()
        self.theta = theta
    def forward(self, output_list, y_list):
        theta = torch.tensor(self.theta, dtype= torch.float32).to(device) 
        loss = 0
        index = 0
        for output in output_list:  
            y = torch.tensor(y_list[index]).to(device).view(-1,1) 
            v = torch.mm(output, theta.view(-1,1)) 
            prob = torch.exp(v) / (torch.sum(torch.exp(v)))  
            loss += torch.sum(-y * torch.log(prob)) 
            index += 1  
        return loss / (len(y_list)/100)


In [4]:
CONTEXT_ARRAY = np.load('features_uni.npy')
theta = np.load('theta_gau.npy')
PURCHASE_LIST = np.load('purchase_uni.npy')

In [7]:
feature_list = []
purchase_list = []

model = SimpleNN(input_size, hidden_size, output_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
for t in range(0,10000):
    context = CONTEXT_ARRAY[t]

    purchase_list.append(PURCHASE_LIST[t])

    feature_list.append(np.array(context))

    if t%10 == 1:
        loss_function = customloss(theta)
        epochs = 1
        for epoch in range(epochs):
            
            output_list = [model(torch.tensor(a,dtype = torch.float32).to(device)) for a in feature_list]
            loss = loss_function(output_list, purchase_list)
     
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
         
            print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 82.71349334716797
epoch 0, loss 96.50592803955078
epoch 0, loss 112.60671997070312
epoch 0, loss 111.05846405029297
epoch 0, loss 98.6337890625
epoch 0, loss 105.79174041748047
epoch 0, loss 108.63611602783203
epoch 0, loss 104.8917007446289
epoch 0, loss 106.26055908203125
epoch 0, loss 99.74435424804688
epoch 0, loss 97.54106140136719
epoch 0, loss 99.97669219970703
epoch 0, loss 100.37325286865234
epoch 0, loss 97.82245635986328
epoch 0, loss 95.50202178955078
epoch 0, loss 94.88429260253906
epoch 0, loss 93.7434310913086
epoch 0, loss 92.20997619628906
epoch 0, loss 91.27852630615234
epoch 0, loss 91.02237701416016
epoch 0, loss 90.10570526123047
epoch 0, loss 90.29621124267578
epoch 0, loss 91.49471282958984
epoch 0, loss 91.43254089355469
epoch 0, loss 91.50299072265625
epoch 0, loss 91.1216812133789
epoch 0, loss 90.50312042236328
epoch 0, loss 92.89944458007812
epoch 0, loss 91.93199920654297
epoch 0, loss 90.56450653076172
epoch 0, loss 89.53015899658203
epoch 0,

KeyboardInterrupt: 

In [14]:
treu = [torch.tensor(a,dtype = torch.float32).to(device) for a in feature_list]
loss_function(treu,purchase_list)

tensor(86.7751, device='cuda:0')

In [ ]:
loss_function()

In [9]:
problist  = np.load('prob_list.npy')
def hhh(i):
    ff = model(torch.tensor(CONTEXT_ARRAY[i],dtype = torch.float32).to(device))
    ff = ff.cpu().detach().numpy()
    u = (ff @ theta.T ).reshape(1,-1)
    prob = np.exp(u) / (np.sum(np.exp(u)))
    
    print(problist[i])
    print(prob)

hhh(1000)   

[2.85832764e-04 1.61504318e-03 1.71842395e-02 3.13120681e-01
 3.59824194e-04 4.86140128e-01 1.81242808e-01 5.14427929e-05]
[[1.59856226e-04 1.05465866e-03 1.37029243e-02 3.13274027e-01
  2.10107517e-04 4.97273834e-01 1.74299437e-01 2.51556344e-05]]
